In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
%pip install opencv-python-headless face-recognition numpy pandas ffmpeg-python


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 17.5 MB/s eta 0:00:00:00:010:01
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566164 sha256=e2f5b3a213db907a8bdad536a3868667482ae5c040a5fb05ea19a35741b56be7
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models
Note: you may need to restart the kernel to use updated packages.


In [20]:
import cv2
import face_recognition
import json
import os
from typing import List, Dict


def extract_faces_from_video(video_path: str, reference_image_path: str, output_dir: str):
    os.makedirs(output_dir, exist_ok=True)
    images_dir = os.path.join(output_dir, 'images')
    os.makedirs(images_dir, exist_ok=True)
    video = cv2.VideoCapture(video_path)
    ref_image = face_recognition.load_image_file(reference_image_path)
    ref_encoding = face_recognition.face_encodings(ref_image)[0]
    frame_rate = int(video.get(cv2.CAP_PROP_FPS))
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    metadata = []
    current_clip_frames = []
    frame_count = 0
    clip_count = 0
    target_face_last_coords = None
    video_writer = None
    success, frame = video.read()
    while success:
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)
        matched = False
        for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
            matches = face_recognition.compare_faces([ref_encoding], face_encoding, tolerance=0.6)
            if matches[0]:
                matched = True
                target_face_coords = [left, top, right - left, bottom - top]
                current_clip_frames.append({
                    "timestamp": frame_count / frame_rate,
                    "bounding_box": target_face_coords
                })
                cropped_face = frame[top:bottom, left:right]
                face_image_path = os.path.join(images_dir, f"frame_{frame_count}.jpg")
                cv2.imwrite(face_image_path, cropped_face)
                if video_writer is None:
                    clip_path = os.path.join(output_dir, f"clip_{clip_count}.mp4")
                    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
                    video_writer = cv2.VideoWriter(clip_path, fourcc, frame_rate, (frame_width, frame_height))
                video_writer.write(frame)
                if target_face_last_coords and has_scene_changed(target_face_last_coords, target_face_coords):
                    finalize_clip(video_writer, current_clip_frames, clip_count, metadata, output_dir)
                    video_writer = None
                    current_clip_frames = []
                    clip_count += 1
                target_face_last_coords = target_face_coords
        if not matched and video_writer is not None:
            finalize_clip(video_writer, current_clip_frames, clip_count, metadata, output_dir)
            video_writer = None
            current_clip_frames = []
            clip_count += 1
        frame_count += 1
        success, frame = video.read()
    if video_writer is not None:
        finalize_clip(video_writer, current_clip_frames, clip_count, metadata, output_dir)
    metadata_path = os.path.join(output_dir, "metadata.json")
    with open(metadata_path, "w") as f:
        json.dump(metadata, f, indent=4)
    video.release()
    return metadata

def has_scene_changed(last_coords: List[int], current_coords: List[int], threshold: float = 0.5) -> bool:
    x1, y1, w1, h1 = last_coords
    x2, y2, w2, h2 = current_coords
    overlap = max(0, min(x1 + w1, x2 + w2) - max(x1, x2)) * max(0, min(y1 + h1, y2 + h2) - max(y1, y2))
    area1 = w1 * h1
    area2 = w2 * h2
    return (overlap / area1 < threshold) or (overlap / area2 < threshold)

def finalize_clip(video_writer, clip_frames, clip_id, metadata, output_dir):
    video_writer.release()
    clip_path = os.path.join(output_dir, f"clip_{clip_id}.mp4")
    start_time = clip_frames[0]["timestamp"]
    end_time = clip_frames[-1]["timestamp"]
    metadata.append({
        "file_name": os.path.basename(clip_path),
        "start_time": start_time,
        "end_time": end_time,
        "frame_coordinates": [frame["bounding_box"] for frame in clip_frames]
    })
    print(f"Finalized clip: {clip_path} from {start_time} to {end_time}")

In [21]:
video_file = "/kaggle/input/video-image-recognize/SIDEMEN 100000 MARIO KART RACE.mp4"
reference_image = "/kaggle/input/video-image-recognize/miniminter.jpg"
output_directory = "/kaggle/working"

metadata = extract_faces_from_video(video_file, reference_image, output_directory)
print(f"Processing completed. Metadata saved in {output_directory}/metadata.json")

Finalized clip: /kaggle/working/clip_0.mp4 from 66.33333333333333 to 66.33333333333333
Finalized clip: /kaggle/working/clip_1.mp4 from 66.43333333333334 to 66.43333333333334
Finalized clip: /kaggle/working/clip_2.mp4 from 66.73333333333333 to 66.73333333333333
Finalized clip: /kaggle/working/clip_3.mp4 from 67.63333333333334 to 67.63333333333334
Finalized clip: /kaggle/working/clip_4.mp4 from 67.7 to 67.76666666666667
Finalized clip: /kaggle/working/clip_5.mp4 from 68.63333333333334 to 68.63333333333334
Finalized clip: /kaggle/working/clip_6.mp4 from 69.03333333333333 to 69.03333333333333
Finalized clip: /kaggle/working/clip_7.mp4 from 69.2 to 69.2
Finalized clip: /kaggle/working/clip_8.mp4 from 69.36666666666666 to 69.4
Finalized clip: /kaggle/working/clip_9.mp4 from 69.93333333333334 to 69.96666666666667
Finalized clip: /kaggle/working/clip_10.mp4 from 70.1 to 70.1
Finalized clip: /kaggle/working/clip_11.mp4 from 70.23333333333333 to 70.23333333333333
Finalized clip: /kaggle/working/

In [22]:
import os
import zipfile

# Specify the directory containing the files
output_dir = '/kaggle/working'

# Create a ZIP file to store the files
zip_filename = '/kaggle/working/output_files.zip'

# List of file extensions to include in the ZIP file
file_extensions = ['.mp4', '.json']

# Create a ZipFile object for writing
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Walk through the directory
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            # Add .mp4 and .json files to the zip
            if file.endswith(tuple(file_extensions)):
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, output_dir))
    
    # Optionally add directories as well (if needed)
    for dir in dirs:
        dir_path = os.path.join(root, dir)
        zipf.write(dir_path, os.path.relpath(dir_path, output_dir))

print(f"All files have been zipped to {zip_filename}")


All files have been zipped to /kaggle/working/output_files.zip


In [23]:
import shutil

# Specify the folder you want to zip
folder_path = '/kaggle/working/images'  # Replace with your folder path

# Specify the output zip file path
zip_file_path = '/kaggle/working/images.zip'

# Create a zip file of the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

print(f"Folder '{folder_path}' has been zipped to '{zip_file_path}'")


Folder '/kaggle/working/images' has been zipped to '/kaggle/working/images.zip'
